In [1]:
!export CUDA_VISIBLE_DEVICES=5

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "5"

In [3]:
import tensorflow as tf
from transformers import BertTokenizerFast, TFAutoModelForTokenClassification
import pandas as pd 
import torch
from sklearn.model_selection import train_test_split
import numpy as np
import re

2024-07-12 14:58:46.012563: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-12 14:58:46.042953: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-12 14:58:46.042983: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-12 14:58:46.061848: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-12 14:58:46.880610: W tensorflow/compiler/tf

In [4]:
if torch.cuda.is_available():
    print("CUDA is available. Number of GPUs:", torch.cuda.device_count())
    print("CUDA device name:", torch.cuda.get_device_name(0))
else:
    print("CUDA is not available.")
torch.cuda.set_device(torch.device("cuda:0"))

CUDA is available. Number of GPUs: 1
CUDA device name: NVIDIA GeForce GTX 1080 Ti


In [5]:
tokenizer = BertTokenizerFast.from_pretrained("google-bert/bert-base-uncased")
model = TFAutoModelForTokenClassification.from_pretrained("bert-base-uncased", num_labels=5)

/home/rpierson/anaconda3/envs/secondenvi/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/rpierson/anaconda3/envs/secondenvi/lib/python3.11/site-packages/transformers/utils/generic.py:319: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
2024-07-12 14:58:50.628810: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10394 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:85:00.0, compute capability: 6.1
All PyTorch model weights were used when initializing TFBertForTokenClassification.

Some weights or buffers of the TF 2.0 model TFBertForTokenCl

In [6]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = [tf.keras.metrics.SparseCategoricalAccuracy()]


In [7]:
strategy = tf.distribute.MirroredStrategy()
if torch.cuda.is_available():
        print(f"LOG: Torch allocated Memory: \
                {torch.cuda.memory_allocated() / 1024 ** 2:.2f} MB", flush=True)
        print(f"LOG: Torch cached Memory: \
                {torch.cuda.memory_reserved() / 1024 ** 2:.2f} MB", flush=True)

if tf.config.experimental.list_physical_devices('GPU'):
    zero_info = tf.config.experimental.get_memory_info('GPU:0')
    print(f"LOG: TensorFlow, 0: Current memory usage: \
            {zero_info['current'] / 1024 ** 2:.2f} MB", flush=True)
    print(f"LOG: TensorFlow, 0: Peak memory usage: \
            {zero_info['peak'] / 1024 ** 2:.2f} MB", flush=True)

# model.compile(optimizer=Adam(3e-5))  # No loss argument!

model.compile(optimizer='Ftrl', loss='sparse_categorical_crossentropy', metrics = ['Accuracy', 'Precision', 'Recall'])

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
LOG: Torch allocated Memory:                 0.00 MB
LOG: Torch cached Memory:                 0.00 MB
LOG: TensorFlow, 0: Current memory usage:             416.42 MB
LOG: TensorFlow, 0: Peak memory usage:             505.58 MB


In [8]:
#Now working with the text

In [9]:
df = pd.read_csv(f'/home/rpierson/Topic_Files/topic_0.csv')
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

In [10]:
def tokenize_function(text):
    return tokenizer(text, padding="max_length", truncation=True)


In [11]:
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)

In [12]:
# Apply the tokenize function to the 'Combined_Text' column
#for x in range(len(train_df)):
#    train_df.loc[x, 'Combined_Text'] = tokenize_function(train_df.loc[x, 'Combined_Text'])


train_df['Combined_Text'] = train_df['Combined_Text'].apply(tokenize_function)
val_df['Combined_Text'] = val_df['Combined_Text'].apply(tokenize_function)

In [13]:
# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_inputs),
    train_labels
)).shuffle(len(train_df)).batch(15)

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_inputs),
    val_labels
)).batch(15)

NameError: name 'train_inputs' is not defined

In [ ]:
model = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=15  # Adjust the number of epochs based on your needs
)